# Load Data

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create an Azure AI Search index and load data](#1)
- [2️⃣ Create a Postgres database and load data](#2)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [7]:
import os
import random

import psycopg2
from dotenv import load_dotenv
from faker import Faker

load_dotenv("../python/.env", override=True)

True

<a id='2'></a>
### 2️⃣ Load data to Postgres

In [ ]:
# Initialize Faker
fake = Faker()

# Database connection parameters from environment variables
db_params = {
    "database": os.getenv("AZURE_POSTGRES_DATABASE"),
    "user": f"{os.getenv('AZURE_POSTGRES_USER')}",
    "password": os.getenv("AZURE_POSTGRES_PASSWORD"),
    "host": f"{os.getenv('AZURE_POSTGRES_SERVER')}", 
    "port": 5432,
    "sslmode": "require"
}

# Connect to the PostgreSQL database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Create the detallespoliza table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS bank_transactions (
    transaction_id SERIAL PRIMARY KEY,
    account_number VARCHAR(20),
    customer_name VARCHAR(255),
    transaction_date DATE,
    transaction_type VARCHAR(10),
    amount DECIMAL(10,2),
    description TEXT
);
"""
cur.execute(create_table_query)
conn.commit()

insert_query = """
INSERT INTO bank_transactions (
    account_number, customer_name, transaction_date, transaction_type, amount, description
) VALUES (
    %s, %s, %s, %s, %s, %s
);
"""

for _ in range(1000):
    tran_type = fake.random_element(elements=["debit", "credit"])
    amt = round(random.uniform(10, 5000), 2)
    record = (
        fake.bban(),  # account_numberer
        fake.name(),  # customer_name),  # customer_name
        fake.date_between(start_date="-5y", end_date="today"),  # transaction_dateart_date="-5y", end_date="today"),  # transaction_date
        tran_type,  # transaction_type
        amt,  # amount
        fake.sentence()  # description
    )
    cur.execute(insert_query, record)

conn.commit()
cur.close()
conn.close()

IndexError: tuple index out of range